# Creating the Channel Funding Transaction

In this section, we’ll create a Lightning channel funding transaction from scratch using Python. We’ll walk through each part of the transaction—how it’s constructed, signed, and how peers exchange messages to share the necessary information to make it happen. We'll test everything using Bitcoin Core in regtest mode.


## Setup

For this exercise we'll need Bitcoin Core. This notebook has been tested with v29.0

Below, set the paths for:

    The bitcoin core functional test framework directory.
    The directory containing taproot-lightning-channels-workshop.

You'll need to edit these next two lines for your local setup.


In [1]:
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/build/test/functional"
path_to_taproot_workshop = "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop"


### Setup bitcoin core test framework

Start up regtest mode, delete any regtest network history so we are starting from scratch.

In [2]:
import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_taproot_workshop)
from functions import *
from functions.bip_0340_reference import *
from functions.test_framework.musig import generate_musig_key
from functions.test_framework.key import generate_bip340_key_pair 
from functions.test_framework.script import tagged_hash
from functions.test_framework.address import program_to_witness


import json

# Setup our regtest environment
test = TestShell().setup(
    num_nodes=1, 
    setup_clean_chain=True
)

node = test.nodes[0]

# Create a new wallet and address to send mining rewards so we can fund our transactions
node.createwallet(wallet_name='mywallet')
address = node.getnewaddress()

# Generate 101 blocks so that the first block subsidy reaches maturity
result = node.generatetoaddress(nblocks=101, address=address, called_by_framework=True)

# Check that we were able to mine 101 blocks
assert(node.getblockcount() == 101)

2025-08-28T00:41:57.469000Z TestFramework (INFO): PRNG seed is: 1919634799285992877
2025-08-28T00:41:57.471000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_rws8vkbh


### Setup Alice and Bob Funding Outputs

Create and fund Taproot addresses for Alice and Bob so they have funds to open a simple Taproot channel. The UTXO created for Alice will be the input of the Channel Funding Transaction.

In [3]:
# Create a new Alice key pair for channel funding multisig output
alice_funding_privkey, alice_funding_pubkey = generate_bip340_key_pair()
alice_funding_pubkey_b = alice_funding_pubkey.get_bytes()
alice_funding_privkey_b = alice_funding_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_funding_address = program_to_witness(0x01, alice_funding_pubkey_b)
print(f"Alice funding pubkey: {alice_funding_pubkey_b.hex()}")
print(f"Alice funding privkey: {alice_funding_privkey_b.hex()}")
print(f"Alice funding address: {alice_funding_address}")

# Create a new Bob key pair for channel funding multisig output
bob_funding_privkey, bob_funding_pubkey = generate_bip340_key_pair()
bob_funding_pubkey_b = bob_funding_pubkey.get_bytes()
bob_funding_privkey_b = bob_funding_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
bob_funding_address = program_to_witness(0x01, bob_funding_pubkey_b)
print(f"Bob funding pubkey: {bob_funding_pubkey_b.hex()}")
print(f"Bob funding privkey: {bob_funding_privkey_b.hex()}")
print(f"Bob funding address: {bob_funding_address}")

# Send 1 BTC to Alice and Bob
alice_initial_fund = 1
bob_initial_fund = 1
txid_to_spend = node.sendmany("", {alice_funding_address: alice_initial_fund, bob_funding_address: bob_initial_fund})
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

Alice funding pubkey: 7744633cea3c716584cb7e4d91097280f6a12e06304d8f5e61166982d37bada3
Alice funding privkey: 1d217d364ea300c995c9c443c29c15f335dc38ff64dbafb782e40cb3c0b68c22
Alice funding address: bcrt1pwazxx08283cktpxt0exezztjsrm2ztsxxpxc7hnpze5c95mm4k3s2rad28
Bob funding pubkey: 7f37b4bdd392c1d5c5da28bdb60045982b8f4bdb5c4dd8ee03545a0a43f673a9
Bob funding privkey: 7ed6d8a361b9f4db9cb53fcec82a999ed19d2f3c08034e66b0298835fd96f38d
Bob funding address: bcrt1p0ummf0wnjtqat3w69z7mvqz9nq4c7j7mt3xa3msr23dq5slkww5sy4t5jx


## Channel Establishment

The "[Extention Bolt](https://github.com/lightning/bolts/pull/995)" for simple taproot channels (work in progress) defines a pathway to create a channel. The messages defined in [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1) are used to exchange the information needed to create the Channel Funding Transaction, as shown below.


    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+


* (1) open_channel - Alice share its "next_local_nonce"

* (2) accept_channel - Bob share its "next_local_nonce"

* (3) funding_created - Alice samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (4) funding_signed - Bob samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (5) channel_ready - Alice share its "next_local_nonce"

* (6) channel_ready - Bob share its "next_local_nonce"


All the nonces are generated using "NonceGen" algorithm defined in [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki) to ensure it generates nonces in a safe manner.

The pubkeys are sorted using the "KeySort" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To compute the aggregated musig2 public key from the sorted funding_pubkeys use the "KeyAgg" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To construct a musig2 partial signature for the sender's remote commitment use the "Sign" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

### The Channel Funding Transaction

Alice has a 1 BTC UTXO that will serve as the funding transaction input. We’ll now create two outputs:

#### The change output back to Alice

In [4]:

# Create a new Alice key pair output for the change
alice_change_privkey, alice_change_pubkey = generate_bip340_key_pair()
alice_change_pubkey_b = alice_change_pubkey.get_bytes()
alice_change_privkey_b = alice_change_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_change_address = program_to_witness(0x01, alice_change_pubkey_b)
print(f"Alice funding pubkey: {alice_change_pubkey_b.hex()}")
print(f"Alice funding privkey: {alice_change_privkey_b.hex()}")
print(f"Alice funding address: {alice_change_address}")


Alice funding pubkey: 567246d054025dc4e33ad799d096215dd0085d4df12680af6d4a4387e1ac3d1d
Alice funding privkey: 877e5bc8e7da81db57e6a61d4af020c469995b714947da591f590ef7f11cdebd
Alice funding address: bcrt1p2eeyd5z5qfwufce667vap93pthgqsh2d7yngptmdffpc0cdv85ws5e66q4


#### The channel musig funding output

The diagram below illustrates how a Taproot channel funding output is constructed.

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +----------+
                    ---| Pagg + T |
                       +----------+
                         ^      ^
                         |      |  +-----------+
                         |       --| T = t * G |<---------
                         |         +-----------+          |
                         |                                |
           +---------------------------------------+    +---+   +-------------------------------+
           |      generate_musig_key(Pa, Pb)       |    | t | = | TaggedHash ("Taptweak", Pagg) |
           +---------------------------------------+    +---+   +-------------------------------+

In [5]:
# Create a new Alice key pair for channel funding multisig output
privkey_alice_musig, pubkey_alice_musig = generate_bip340_key_pair()
print(f"Alice MuSig pubkey: {pubkey_alice_musig.get_bytes().hex()}")
print(f"Alice MuSig privkey: {privkey_alice_musig.get_bytes().hex()}")

# Create a new Bob key pair for channel funding multisig output
privkey_bob_musig, pubkey_bob_musig = generate_bip340_key_pair()
print(f"Bob MuSig pubkey: {pubkey_bob_musig.get_bytes().hex()}")
print(f"Bob MuSig privkey: {privkey_bob_musig.get_bytes().hex()}")

# Generate a 2-of-2 aggregate MuSig key using the pubkeys form Alice and Bob
c_map, agg_pubkey = generate_musig_key([pubkey_alice_musig, pubkey_bob_musig])

# Multiply individual keys with challenges
privalice_c = c_map[pubkey_alice_musig] * privkey_alice_musig
privbob_c = c_map[pubkey_bob_musig] * privkey_bob_musig
pubalice_c = c_map[pubkey_alice_musig] * pubkey_alice_musig
pubbob_c = c_map[pubkey_bob_musig] * pubkey_bob_musig

if agg_pubkey.get_y()%2 != 0:
    privalice_c.negate()
    privbob_c.negate()
    pubalice_c.negate()
    pubbob_c.negate()
    agg_pubkey.negate()

print(f"Aggregate musig pubkey: {agg_pubkey.get_bytes().hex()}")

# Tweak musig public key
# Method: ECPubKey.tweak_add()
tweak = tagged_hash("TapTweak", agg_pubkey.get_bytes())
agg_pubkey_tweaked = agg_pubkey.tweak_add(tweak)
print(f"Tweaked aggregate musig pubkey: {agg_pubkey_tweaked.get_bytes().hex()}")

agg_pubkey_tweaked_b = agg_pubkey_tweaked.get_bytes()

# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
channel_funding_musig_address = program_to_witness(0x01, agg_pubkey_tweaked_b)
print(f"Channel funding musig output address: {address}")

Alice MuSig pubkey: 4972e2a35737fb75912d35f3f87658940a86654cefe4941d9a21fb4cb7e315f3
Alice MuSig privkey: 98ef3361c074962eddac0f7c11ae9e6cc87d3253236b23b1dcd267b770813194
Bob MuSig pubkey: 57aabede051a9e52e3d257d5ec53a560b53e9a93dc1dcf4c7ec3e84660e51256
Bob MuSig privkey: 61b2ee590b8359c7adeb8f1bd9b34069b70fcd7269c3858087669e19647899ed
Aggregate musig pubkey: b12670f07364191a6f4cca01b35d4415b8d6f59f9a5116f2e7478b2780f6dc25
Tweaked aggregate musig pubkey: 75fc196f4847f59c8b800caf3d4d80c8a8faeccda30d89987187db2eb8c23b97
Channel funding musig output address: bcrt1qr5zph7w8qjkqv3dejjxa8v09ur7dh94jpyqcp9


#### The channel funding transaction

In [ ]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = bytes.fromhex("00000000")

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# Transaction Fee
tx_fee_sat = int(float("0.0001") * 100000000)

# OUTPUT 1 - Channel Funding Output
# 0.01 BTC to the 2-of-2 multisig Taproot output
output1_value_sat = int(float("0.01") * 100000000)
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output1_spk = bytes.fromhex("51") + varint_len(agg_pubkey_tweaked_b) + agg_pubkey_tweaked_b

# OUTPUT 2 - Change Output
# The rest back to Alice's change address
# 1 BTC - 0.01 BTC - 0.0001 BTC = 0.9899 BTC
output2_value_sat = int(float(alice_initial_fund) * 100000000) - output1_value_sat - tx_fee_sat
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output2_spk = bytes.fromhex("51") + varint_len(alice_change_pubkey_b) + alice_change_pubkey_b

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

unsigned_tx:  0200000001b4f8c76cded8a6658b794f0af01cc975c3cfc458c0ff77e4f6db45bcb8b13576000000ffffffff0240420f000000000022512075fc196f4847f59c8b800caf3d4d80c8a8faeccda30d89987187db2eb8c23b97b077e60500000000225120567246d054025dc4e33ad799d096215dd0085d4df12680af6d4a4387e1ac3d1d00000000


JSONRPCException: TX decode failed (-22)